In [41]:
import numpy as np
import matplotlib.pyplot as plt
import datetime
import re
from elasticsearch import Elasticsearch, exceptions as es_exceptions
from elasticsearch.helpers import scan

#Create instance of ElasticSearch using the Atlas host server
es = Elasticsearch(hosts=[{'host':'atlas-kibana.mwt2.org', 'port':9200}],timeout=60)

In [42]:
#Get indices for network_weather-2*
indices = es.cat.indices(index="network_weather_2-*", h="index", request_timeout=600).split('\n')
indices = sorted(indices)
indices = [x for x in indices if x != '']

In [46]:
import ast

_src = "CERN*"
_dst = "BNL-ATLAS*"
field = "delay_mean"
period = '90d'
binSize = "1d"
aggType = "avg"

#def getFieldWithTimestamp(_src,_dst,field,periodStart,periodStop,binSize,aggType):
queryTemplate= '''{
  "query": {
    "filtered": {
      "query": {
        "query_string": {
          "query": "srcSite:%s AND destSite:%s",
          "analyze_wildcard": True,
          "lowercase_expanded_terms": False
        }
      },
      "filter": {
        "bool": {
          "must": [
            {
              "range": {
                "@timestamp": {
                  "gte": 'now - %s',
                  "lte": 'now',
                  "format": "epoch_millis"
                }
              }
            }
          ],
          "must_not": []
        }
      }
    }
  },
  "size": 0,
  "aggs": {
    "2": {
      "date_histogram": {
        "field": "@timestamp",
        "interval": "%s",
        "time_zone": "Europe/Berlin",
        "min_doc_count": 1,
        "extended_bounds": {
          "min": 'now - %s',
          "max": 'now'
        }
      },
      "aggs": {
        "1": {
          "%s": {
            "field": "%s"
          }
        }
      }
    }
  }
}'''
query = queryTemplate%(_src,_dst,period,binSize,period,aggType,field)
query

'{\n  "query": {\n    "filtered": {\n      "query": {\n        "query_string": {\n          "query": "srcSite:CERN* AND destSite:BNL-ATLAS*",\n          "analyze_wildcard": True,\n          "lowercase_expanded_terms": False\n        }\n      },\n      "filter": {\n        "bool": {\n          "must": [\n            {\n              "range": {\n                "@timestamp": {\n                  "gte": \'now - 90d\',\n                  "lte": \'now\',\n                  "format": "epoch_millis"\n                }\n              }\n            }\n          ],\n          "must_not": []\n        }\n      }\n    }\n  },\n  "size": 0,\n  "aggs": {\n    "2": {\n      "date_histogram": {\n        "field": "@timestamp",\n        "interval": "1d",\n        "time_zone": "Europe/Berlin",\n        "min_doc_count": 1,\n        "extended_bounds": {\n          "min": \'now - 90d\',\n          "max": \'now\'\n        }\n      },\n      "aggs": {\n        "1": {\n          "avg": {\n            "field": 

In [47]:
builtQuery = ast.literal_eval(query)
print(builtQuery)
response = es.search(index="network_weather_2-*", body=builtQuery, request_timeout=600)
print(response)
#Initialize lists
fieldVals = []
timestamp = []

#For each bucket:
for r in response['aggregations']['2']['buckets']:

    #Get field value and timestamp
    fieldVals.append(r['1']['value'])
    timestamp.append(r['key'])
 
#Convert both to numpy arrays
timestamp = np.array(timestamp)
fieldVals = np.array(fieldVals)

{'query': {'filtered': {'query': {'query_string': {'lowercase_expanded_terms': False, 'analyze_wildcard': True, 'query': 'srcSite:CERN* AND destSite:BNL-ATLAS*'}}, 'filter': {'bool': {'must_not': [], 'must': [{'range': {'@timestamp': {'gte': 'now - 90d', 'lte': 'now', 'format': 'epoch_millis'}}}]}}}}, 'size': 0, 'aggs': {'2': {'date_histogram': {'field': '@timestamp', 'interval': '1d', 'extended_bounds': {'min': 'now - 90d', 'max': 'now'}, 'min_doc_count': 1, 'time_zone': 'Europe/Berlin'}, 'aggs': {'1': {'avg': {'field': 'delay_mean'}}}}}}
{'timed_out': False, 'took': 96, 'hits': {'total': 0, 'hits': [], 'max_score': 0.0}, '_shards': {'failed': 0, 'total': 925, 'successful': 925}, 'aggregations': {'2': {'buckets': []}}}


In [39]:
timestamp

array([], dtype=float64)

TypeError: getFieldWithTimestamp() missing 1 required positional argument: 'aggType'

In [27]:
q

{'aggs': {'2': {'aggs': {'1': {'avg': {'field': 'delay_mean'}}},
   'date_histogram': {'extended_bounds': {'max': 1468241030907,
     'min': 1452519830907},
    'field': '@timestamp',
    'interval': '1d',
    'min_doc_count': 1,
    'time_zone': 'Europe/Berlin'}}},
 'query': {'filtered': {'filter': {'bool': {'must': [{'range': {'@timestamp': {'format': 'epoch_millis',
         'gte': 1452519830907,
         'lte': 1468241030907}}}],
     'must_not': []}},
   'query': {'query_string': {'analyze_wildcard': True,
     'lowercase_expanded_terms': False,
     'query': 'srcSite:CERN* AND destSite:BNL*'}}}},
 'size': 0}